#### <b> ML Interoperability with ONNX on Vertex AI: </b>
 1. Convert BQML/AutoML model into a tensorflow package (via GUI) and call the tensorflow model via tensorflow serving client
 2. Convert Tensorflow model into ONNX 

In [14]:
# Export BQML model into GCS.
!gsutil ls "gs://bonibon/model/model-export/fraud/bqml-reg"

gs://bonibon/model/model-export/fraud/bqml-reg/
gs://bonibon/model/model-export/fraud/bqml-reg/saved_model.pb
gs://bonibon/model/model-export/fraud/bqml-reg/variables/


In [20]:
# Read the BQML model signature for serving.
!saved_model_cli show --dir gs://bonibon/model/model-export/fraud/bqml-reg --tag_set serve

The given SavedModel MetaGraphDef contains SignatureDefs with the following keys:
SignatureDef key: "serving_default"


In [15]:
# Call tensorfow serving - option1
import tensorflow as tf
path="gs://bonibon/model/model-export/fraud/bqml-reg"
#path="gs://bonibon/model/model-export/pmml/onnx_test_automl/model-6930059062151217152/tf-saved-model/2021-11-08T15:35:56.620287Z/predict/001"
imported = tf.saved_model.load(path)
f = imported.signatures["serving_default"]
print(f(oldbalanceOrig=tf.constant([[1.]]),oldbalanceDest=tf.constant([[1.]]), amount=tf.constant([[1.]])))

{'isFraud_values': <tf.Tensor: id=2247, shape=(1, 2), dtype=string, numpy=array([[b'1', b'0']], dtype=object)>, 'isFraud_probs': <tf.Tensor: id=2246, shape=(1, 2), dtype=float64, numpy=array([[0.0155844, 0.9844156]])>, 'predicted_isFraud': <tf.Tensor: id=2248, shape=(1, 1), dtype=string, numpy=array([[b'0']], dtype=object)>}


In [17]:
# Call tensorfow serving - option2
!saved_model_cli run --dir "gs://bonibon/model/model-export/fraud/bqml-reg/" --tag_set serve --signature_def "serving_default" --input_exprs 'oldbalanceDest=[0.01];amount=[0.0];oldbalanceOrig=[0.1]' 

2021-11-08 19:07:14.689055: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
2021-11-08 19:07:14.697406: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2199995000 Hz
2021-11-08 19:07:14.697842: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x55a77e2851e0 executing computations on platform Host. Devices:
2021-11-08 19:07:14.697895: I tensorflow/compiler/xla/service/service.cc:175]   StreamExecutor device (0): Host, Default Version
Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.loader.load or tf.compat.v1.saved_model.load. There will be a new function for importing SavedModels in Tensorflow 2.0.
Result for output key isFraud_probs:
[[0.01558435 0.98441565]]
Result for output key isFraud_values:
[[b'1' b'0']]
Result for output key predicted_isFraud:
[[b'0']]


https://www.tensorflow.org/tfx/guide/serving

In [21]:
#!pip install -U tf2onnx

In [18]:
# Convert tensorflow model into ONNX.
!python -m tf2onnx.convert --saved-model "gs://bonibon/model/model-export/fraud/bqml-reg" --output model.onnx --tag serve

/opt/conda/lib/python3.7/runpy.py:125: RuntimeWarning: 'tf2onnx.convert' found in sys.modules after import of package 'tf2onnx', but prior to execution of 'tf2onnx.convert'; this may result in unpredictable behaviour
  warn(RuntimeWarning(msg))
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
2021-11-08 19:07:31,721 - WARNING - From /opt/conda/lib/python3.7/site-packages/tensorflow_core/python/ops/resource_variable_ops.py:1781: calling BaseResourceVariable.__init__ (from tensorflow.python.ops.resource_variable_ops) with constraint is deprecated and will be removed in a future version.
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
2021-11-08 19:07:32,766 - INFO - Signatures found in model: [serving_default].
2021-11-08 19:07:32,766 - WARNING - '--signature_def' not specified, using first signature: serving_default
2021-11-08 19:07:32,766 - INFO - Output names: ['isFraud_probs', 'isFraud_values', 'predicted_isFraud']
Type

In [19]:
!ls -la

total 60
drwxr-xr-x  4 jupyter jupyter  4096 Nov  8 19:06 .
drwxr-xr-x 14 jupyter jupyter  4096 Nov  4 20:27 ..
drwxr-xr-x  2 jupyter jupyter  4096 Nov  8 17:48 .ipynb_checkpoints
drwxr-xr-x  2 jupyter jupyter  4096 Nov  4 20:37 model
-rw-r--r--  1 jupyter jupyter  5937 Nov  8 19:07 model.onnx
-rw-r--r--  1 jupyter jupyter 12171 Nov  8 17:49 vertex-onnx-conversion-automl.ipynb
-rw-r--r--  1 jupyter jupyter 12119 Nov  8 19:06 vertex-onnx-conversion-bqml-reg-Copy1.ipynb
-rw-r--r--  1 jupyter jupyter 11532 Nov  8 15:43 vertex-onnx-conversion-bqml-reg.ipynb


In [69]:
# The end.